In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['Times New Roman']
plt.rcParams['figure.dpi'] = 500
plt.rcParams['text.color'] = 'w'
plt.rcParams['axes.labelcolor'] = 'k'
plt.rcParams['xtick.color'] = 'w'
plt.rcParams['ytick.color'] = 'w'
plt.rcParams['axes.edgecolor'] = 'w'
plt.rcParams['figure.facecolor'] = 'k'
plt.rcParams['axes.facecolor'] = 'k'
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.colors import LogNorm
from matplotlib.patches import FancyArrowPatch
import matplotlib.colors as colors
from astropy.visualization import SqrtStretch, LinearStretch, LogStretch
from astropy.visualization.mpl_normalize import ImageNormalize
import pandas as pd
import re
from astropy.io import fits
import os
from scipy.ndimage import map_coordinates
from scipy.stats import binned_statistic
from scipy.spatial.distance import cdist
from scipy.optimize import linear_sum_assignment
from pathlib import Path
from scipy.stats import gaussian_kde
import seaborn as sns
import plotly.graph_objects as go

from matplotlib.lines import Line2D
os.chdir("/Users/ainsleylewis/Documents/Astronomy/IllustrisTNG Lens Modelling")

In [ ]:
df = pd.read_csv('/Users/ainsleylewis/Documents/Astronomy/IllustrisTNG Lens Modelling/Test/Simulation/sim_3.csv', header=0)

In [ ]:
# Fill missing values in pos_rms column with 1
df['pos_rms'] = df['pos_rms'].fillna(1)
df['mag_rms'] = df['mag_rms'].fillna(6000)

In [ ]:
from scipy.interpolate import griddata

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Create a grid of x and y values
x = df['t_shear_str'].values
y = df['t_shear_kappa'].values
z = df['pos_rms'].values

# Remove NaN values
mask = ~np.isnan(z)
x = x[mask]
y = y[mask]
z = z[mask]

# Create interpolation grid
xi = np.linspace(x.min(), x.max(), 100)
yi = np.linspace(y.min(), y.max(), 100)
xi, yi = np.meshgrid(xi, yi)

# Interpolate z values on regular grid
zi = griddata((x, y), z, (xi, yi), method='cubic')

# Plot surface
surf = ax.plot_surface(xi, yi, zi, cmap='magma', alpha=1)

# Plot scattered points
scatter = ax.scatter(x, y, z, c=z, cmap='magma', alpha=0)

ax.set_xlabel('Shear Strength', color = 'w')
ax.set_ylabel('Shear Kappa', color = 'w')
ax.set_zlabel('Position RMS', color = 'w')

fig.colorbar(surf, label='Position RMS', ax=ax)
plt.title('3D Plot of Shear Parameters vs Position RMS', color = 'w')
plt.tight_layout()
plt.show()

In [ ]:
# Create the contour plot
plt.figure(figsize=(12, 8))

# Create the contour plot using the same data
contour = plt.contourf(xi, yi, zi, levels=50, cmap='magma')

# Add a colorbar
plt.colorbar(contour, label='Position RMS')

# Set labels and title
plt.xlabel('Shear Strength', color = 'w')
plt.ylabel('Shear Kappa', color = 'w')
plt.title('Shear Strength and Kappa Parameter Space', color = 'w')

# Set the style for dark theme
plt.style.use('dark_background')
plt.grid(True, alpha=0.2)

# Adjust layout
plt.tight_layout()

# Show the plot
plt.show()


In [ ]:
one_sigma = 0.001
sigma_levels = [1, 5, 10]

# Create a figure with subplots
plt.figure(figsize=(12, 8))

for i in range(len(sigma_levels)):
    # Create a subplot for each sigma level
    ax = plt.subplot(1, len(sigma_levels), i + 1)
    
    # Filter data based on sigma level
    mask = (df['pos_rms'] <= one_sigma * sigma_levels[i])
    filtered_df = df[mask]
    
    # Create grid for contour plot
    xi = np.linspace(filtered_df['t_shear_str'].min(), filtered_df['t_shear_str'].max(), 100)
    yi = np.linspace(filtered_df['t_shear_kappa'].min(), filtered_df['t_shear_kappa'].max(), 100)
    xi, yi = np.meshgrid(xi, yi)
    
    # Interpolate z values
    zi = griddata((filtered_df['t_shear_str'], filtered_df['t_shear_kappa']), 
                 filtered_df['pos_rms'], (xi, yi), method='cubic')
    
    # Create contour plot
    ax.contourf(xi, yi, zi, levels=50, cmap='magma')
    ax.set_title(f'{sigma_levels[i]} $\sigma$', color='w')
    ax.set_xlabel('Shear Strength', color='w')
    ax.set_ylabel('Shear Kappa', color='w')
    ax.set_facecolor('black')
    ax.tick_params(colors='w')
    ax.grid(True, alpha=0.2)

plt.tight_layout()
plt.show()